In [1]:
import os
import json
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import train_test_split

In [2]:
import torchvision.models as models
import torch.nn as nn
num_classes = 4
resnet = models.resnet50(pretrained=True)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

c:\Users\user\anaconda3\envs\torch-book\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\torch-book\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=4)  # 여기에 사용한 모델 구조와 동일해야 합니다.
model.load_state_dict(torch.load('C:/SS/AI_pj/best_model_weights.pth', map_location=device))
model.to(device)
model.eval()

C:\Users\user\AppData\Local\Temp\ipykernel_13972\393387443.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('C:/SS/AI_pj/best_model_weigh

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=4, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 라벨 맵핑 (예시)
label_map = {0: "아우터", 1: "하의", 2: "원피스", 3: "상의"}

# 예측 함수
def predict_image(image_path, model, transform, label_map):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # 배치 차원 추가

    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    predicted_label = label_map[predicted.item()]
    return predicted_label

In [8]:
import os
from tqdm import tqdm  # tqdm 모듈 임포트

top_folder = r'C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim'
count, accuracy = 0, 0
max_iterations = 1000  # 최대 반복 횟수 설정
correct_predictions = []  # 정답을 맞춘 이미지 파일 경로를 저장할 리스트

# 최상위 폴더 하위의 모든 폴더와 파일을 순회
for root, dirs, files in os.walk(top_folder):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(root, file)
            
            # 예측 수행
            predicted_label = predict_image(image_path, model, transform, label_map)
            
            # 파일명에 예측된 라벨이 포함되어 있는지 확인
            if predicted_label in file:
                accuracy += 1
                correct_predictions.append(image_path)  # 정답을 맞춘 이미지 경로 저장

            count += 1 
            
            # 최대 반복 횟수에 도달하면 중단
            if count >= max_iterations:
                break
    
    if count >= max_iterations:
        break

# 정확도 계산 및 출력
if count > 0:
    print(f"Accuracy: {accuracy / count * 100:.2f}%")
    print(f"Correct predictions: {len(correct_predictions)}")
else:
    print("No images processed.")

# 정답을 맞춘 이미지 파일 목록 출력
if correct_predictions:
    print("\nCorrectly predicted images:")
    for img_path in correct_predictions:
        print(img_path)
else:
    print("No correct predictions.")


Accuracy: 5.70%
Correct predictions: 57

Correctly predicted images:
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\10\10_원피스.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\100\100_원피스.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\1000\1000_원피스.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\1001\1001_원피스.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\1002\1002_원피스.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\1003\1003_원피스.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\10031\10031_하의.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\1005\1005_원피스.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\10055\10055_하의.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_HardSegImgs_aim\1006\1006_원피스.jpg
C:\SS\AI_project_data\kfashoin_ai_model\img\K-fashion_H